<a href="https://colab.research.google.com/github/PranavkrishnaVadhyar/Knowledge-Navigator/blob/main/Knowledge_navigator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chunking

In [1]:
!pip install langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.5/366.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.6/987.6 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 4.6 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade --quiet  langchain sentence_transformers
!pip install langchain
!pip install langchain_community

In [4]:
with open("/content/drive/MyDrive/test.txt","r",encoding='utf-8') as f:
    data = f.read()



In [5]:
data

In [6]:
from google.colab import userdata
key = userdata.get('HF_TOKEN')

In [7]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings.spacy_embeddings import SpacyEmbeddings


text_splitter = SemanticChunker(HuggingFaceBgeEmbeddings())

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [9]:
docs = text_splitter.create_documents([data])



In [10]:
docs

[Document(page_content='Phase 1 \n\nMarket research and market compatibility testing: \n\nPerforming market survey for product.'),
 Document(page_content='Involves interviewing potential users and medical professionals. Helps understanding target audience and potential markets\n\nData collection and data preparation \n\nBuilding on existing MVP by increasing training data\nOptimizing data preparation methods to increase model performance\n\n\nPhase 2\n\nModel research \n\nResearching models compatible for current use case of classification\n\nModel training\n\nTraining the selected model with prepared data\nEvaluating model performance\nComparison of model performance with benchmarked models\n\nDeployment of model \n\nDeployment of model using Flask API\nAPI testing\n\nPhase 3\n\nLLM Integration\n\nResearching Large language models for treatment recommendation\nResearching strategies for LLM usage : API, Fine-tuning, Retrieval augmented generation(RAG)\nAnalysing LLM performance\n\nImp

In [11]:
len(docs)

2

#MILVUS Vector store

In [12]:
!pip install -U pymilvus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 14.2 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.64.1
    Uninstalling grpcio-1.64.1:
      Successfully uninstalled grpcio-1.64.1


In [7]:
from pymilvus import MilvusClient

client = MilvusClient("/content/test.db")

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 09ee0884b0b44b1db0f51ec746b1d9b8


In [ ]:
if client.has_collection(collection_name="demo_collection"):
    client.drop_collection(collection_name="demo_collection")
client.create_collection(
    collection_name="demo_collection",
    dimension=768,  # vector dimension
    metric_type="L2",  # vector distance metric
    index_params={"index_type": "HNSW"},
)


In [ ]:
!pip install "pymilvus[model]"


In [ ]:
from pymilvus import model

embedding_fn = model.DefaultEmbeddingFunction()

# Extract the text content from the Document objects
texts = [doc.page_content for doc in docs]

vectors = embedding_fn.encode_documents(texts)
print("Dim:", embedding_fn.dim, vectors[0].shape)  # Dim: 768 (768,)

data = [
    {"id": i, "vector": vectors[i], "text": texts[i], "subject": "history"}
    for i in range(len(vectors))
]

print("Data has", len(data), "entities, each with fields: ", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))


In [ ]:
res = client.insert(collection_name="demo_collection", data=data)  # Use 'data' instead of 'docs'

print(res)




#Retrieval and Re-ranking

In [2]:
!pip install nltk
import nltk

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
!pip install rank_bm25

In [1]:

from pymilvus import connections, Collection, MilvusClient
from transformers import AutoModel, AutoTokenizer
from rank_bm25 import BM25Okapi
import torch


client = MilvusClient("/content/test.db")

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Define function to get BERT embeddings for a query
def get_query_embedding(query):
    inputs = tokenizer(query, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# BM25 setup with tokenized documents (you need to prepare this)
documents = [["example", "document", "1"], ["example", "document", "2"]]  # replace with your tokenized documents
bm25 = BM25Okapi(documents)

# BM25 retrieval function
def bm25_retrieve(query):
    tokenized_query = tokenizer.tokenize(query)
    return bm25.get_scores(tokenized_query)

# Vector-based retrieval using Milvus
collection = Collection("demo_collection")


def vector_retrieve(query):
    query_embedding = get_query_embedding(query)
    search_params = {
        "metric_type": "L2",
        "params": {"nprobe": 10},
    }
    results = collection.search(
        data=[query_embedding],
        anns_field="embedding",
        param=search_params,
        limit=10,
        expr=None,
    )
    return results

# Hybrid retrieval combining BM25 and vector-based methods
def hybrid_retrieve(query):
    bm25_scores = bm25_retrieve(query)
    vector_results = vector_retrieve(query)

    combined_results = []
    for idx, result in enumerate(vector_results):
        combined_score = bm25_scores[idx] + result.distance  # Normalize if needed
        combined_results.append((result.id, combined_score))

    combined_results.sort(key=lambda x: x[1], reverse=True)
    return combined_results

# Re-rank the retrieved data
def re_rank_results(results, query):
    ranked_results = sorted(results, key=lambda x: x[1], reverse=True)
    return ranked_results

# Example usage
query = "your search query"
expanded_query = "expanded " + query  # Implement your query expansion logic
retrieved_data = hybrid_retrieve(expanded_query)
ranked_data = re_rank_results(retrieved_data, query)

for data in ranked_data:
    print(data)


'\nfrom pymilvus import connections, Collection, MilvusClient\nfrom transformers import AutoModel, AutoTokenizer\nfrom rank_bm25 import BM25Okapi\nimport torch\n\n\nclient = MilvusClient("/content/test.db")\n\n# Load pre-trained BERT model and tokenizer\nmodel_name = \'bert-base-uncased\'\ntokenizer = AutoTokenizer.from_pretrained(model_name)\nmodel = AutoModel.from_pretrained(model_name)\n\n# Define function to get BERT embeddings for a query\ndef get_query_embedding(query):\n    inputs = tokenizer(query, return_tensors=\'pt\')\n    with torch.no_grad():\n        outputs = model(**inputs)\n    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()\n\n# BM25 setup with tokenized documents (you need to prepare this)\ndocuments = [["example", "document", "1"], ["example", "document", "2"]]  # replace with your tokenized documents\nbm25 = BM25Okapi(documents)\n\n# BM25 retrieval function\ndef bm25_retrieve(query):\n    tokenized_query = tokenizer.tokenize(query)\n    return bm25.g

#ChatBot

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Milvus
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from pymilvus import MilvusClient
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize



# Initialize embeddings and LLM
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-mpnet-base-v2")
llm = OpenAI(temperature=0.7)

# Example query
query = input("Enter query: ")

retrieved_data = hybrid_retrieve(query)
ranked_data = re_rank_results(retrieved_data, query)


# Format documents into a context string
context = "Context for the question:\n\n"
for doc_title, doc_content in ranked_data:
    context += f"{doc_title}:\n{doc_content}\n\n"

print("Formatted Context:")
print(context)



In [ ]:
while True:
    # Example question
    question = input("Enter your question")

    if question == "exit":
        break

    # Initialize LangChain LLMChain
    llm_chain = LLMChain(llm)

    # Generate answer
    answer = llm_chain.answer(question, context)
    print("\nAnswer from LLM:")
    print(answer)